In [12]:
import ee
import geemap

In [14]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [15]:
# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
Map.addLayer(image, vis_params, 'SRTM DEM')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')

In [16]:
Map.draw_features

[]

In [17]:
Map.draw_last_feature

In [18]:
roi = ee.FeatureCollection(Map.draw_features)
selected_states = states.filterBounds(roi)
Map.addLayer(selected_states, {}, "Selected states")

In [19]:
clipped_image = image.clip(selected_states)
Map.addLayer(clipped_image, vis_params, 'Clipped image')